In [3]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import keras_metrics
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
#STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
!pip install keras_metrics

In [7]:
olidData=pd.read_csv('training_data.tsv', sep='\t')
olidData
olidData.text=olidData.text.astype(str)

,id,text,actual_label
0,1157097496465825792,artists use clip studio paint please dm looks ...,NOT
1,1159951246964547584,personORGillORGworkORGORGmightORGneedORGpens,NOT
2,1161854610727997440,person 1 mycareer trailer huh,NOT
3,1187418039719026689,aint settling piece shit ass hoes,"OFF, UNT"
4,1157363369898328064,person person yea 100 part group,NOT
...,...,...,...
134683,1160273231154176005,im done fucking gangbangs dumbass men love lik...,"OFF, UNT"
134684,1162081126774652929,get rid trump asap person,NOT
134685,1165255509202653185,exposing somebody love big l,NOT
134686,1160347194370809861,parents really inconsiderate fuck,"OFF, UNT"


In [8]:
maxWords = 50000
maxLength = 250
embedDimension = 100

tokenizer = Tokenizer(num_words=maxWords, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(olidData['text'])
indexedWords = tokenizer.word_index
print('Found %s unique tokens.' % len(indexedWords))

X_train = tokenizer.texts_to_sequences(olidData['text'].values)
X_train = pad_sequences(X_train, maxlen=maxLength)
print('Shape of data tensor:', X_train.shape)

Y_train = pd.get_dummies(olidData['actual_label'])
print('Shape of label tensor:', Y_train.shape)

print(X_train.shape,Y_train.shape)

Found 77145 unique tokens.
Shape of data tensor: (134688, 250)
Shape of label tensor: (134688, 5)
(134688, 250) (134688, 5)


In [0]:
biDirModel = Sequential()
biDirModel.add(Embedding(maxWords, embedDimension, input_length=X_train.shape[1]))
biDirModel.add(SpatialDropout1D(0.2))
biDirModel.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
biDirModel.add(Dense(5, activation='softmax'))
biDirModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',keras_metrics.precision(),keras_metrics.recall(),keras_metrics.f1_score()])
print(biDirModel.summary())

# BiLSTM model training
epochs = 5
batch_size = 64

history = biDirModel.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 250, 100)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1005      
Total params: 5,161,805
Trainable params: 5,161,805
Non-trainable params: 0
_________________________________________________________________
None
Train on 121219 samples, validate on 13469 samples
Epoch 1/5
 26624/121219 [=====>........................] - ETA: 25:41 - loss: 0.5635 - acc: 0.7948 - precision: 0.8794 - recall: 0.9455 - f1_score: 0.9112

Reading Test Data and then tokenizing test.

In [11]:
testData=pd.read_csv('testData.csv')
testData.tweet=testData.tweet.astype(str)
testData

tokenizer = Tokenizer(num_words=maxWords, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(testData['tweet'])
indexedWords = tokenizer.word_index
print('Found %s unique tokens.' % len(indexedWords))


X_test = tokenizer.texts_to_sequences(testData['tweet'].values)
X_test = pad_sequences(X_test, maxlen=maxLength)
print('Shape of data tensor:', X_test.shape)

Y_test = pd.get_dummies(testData['actual_label'])
print('Shape of label tensor:', Y_test.shape)

accr = biDirModel.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Precision: {:0.3f}\n  Recall: {:0.3f}\n  F1Score: {:0.3f}'.format(accr[0],accr[1],accr[2],accr[3],accr[4]))

,Unnamed: 0,id,tweet,subtask_a,subtask_b,subtask_c,actual_label
0,0,86426,person ask native americans take,OFF,UNT,NaN,"OFF, UNT"
1,1,90194,person person go home youre drunk person #maga...,OFF,TIN,IND,"OFF, TIN, IND"
2,2,16820,amazon investigating chinese employees selling...,NOT,NaN,NaN,NOT
3,3,62688,person someone shouldvetaken piece shit volcano,OFF,UNT,NaN,"OFF, UNT"
4,4,43605,person person obama wanted liberals amp illega...,NOT,NaN,NaN,NOT
...,...,...,...,...,...,...,...
13235,13235,95338,person sometimes get strong vibes people mans ...,OFF,TIN,IND,"OFF, TIN, IND"
13236,13236,67210,benidorm creamfields maga shabby summer,NOT,NaN,NaN,NOT
13237,13237,82921,person report garbage dont give crap,OFF,TIN,OTH,"OFF, TIN, OTH"
13238,13238,27429,person pussy,OFF,UNT,NaN,"OFF, UNT"


Found 20373 unique tokens.
Shape of data tensor: (13240, 250)
Shape of label tensor: (13240, 5)
13240/13240 [==============================] - 48s 4ms/step
Test set
  Loss: 5.216
  Accuracy: 0.365
  Precision: 0.670
  Recall: 0.445
  F1Score: 0.535


In [0]:
import numpy as np

def InsertPredictions(test_data,model):
    lstNot=[]
    lstOffGrp=[]
    lstOffInd=[]
    lstOffOth=[]
    lstOffUnt=[]
    lst=["NOT","OFF, TIN, GRP","OFF, TIN, IND","OFF, TIN, OTH","OFF, UNT"]
    dd=testdata
    lst_text = dd['tweet'].tolist()
    y_prob = model.predict(test_data)
    for prob in y_prob:
      lstNot.append(prob[0])
      lstOffGrp.append(prob[1])
      lstOffInd.append(prob[2])
      lstOffOth.append(prob[3])
      lstOffUnt.append(prob[4])
    y_classes = y_prob.argmax(axis=-1)
    lst_classes=y_classes.tolist()
    lst_predictions = [lst[i] for i in lst_classes]
    lst_original=dd['actual_label'].tolist()
    lst_prob=y_prob.tolist()
    df = pd.DataFrame(lst_text,columns =['tweet'])
    df['Original']=lst_original
    df['Predictions']=lst_predictions
    df['Not Off.']=lstNot
    df['Off. Group']=lstOffGrp
    df['Off. Ind.']=lstOffInd
    df['Off. Others']=lstOffOth
    df['Off. Untargeted']=lstOffUnt
    lstSummary=[]
    for index, row in df.iterrows():
      if row['Original']==row['Predictions']:
        lstSummary.append(1)
      else:
        lstSummary.append(0)
    df['IsCorrect']=lstSummary
    return df

predicted=InsertPredictions(X_test,biDirModel)
predicted

#y_classes = keras.np_utils.probas_to_classes(y_proba)
#y_classes[0][0]

,tweet,Original,Predictions,Not Off.,Off. Group,Off. Ind.,Off. Others,Off. Untargeted,IsCorrect
0,person ask native americans take,"OFF, UNT",NOT,0.999746,1.639387e-04,1.290461e-05,0.000076,5.612267e-07,0
1,person person go home youre drunk person #maga...,"OFF, TIN, IND",NOT,0.999997,4.525802e-07,4.465868e-07,0.000001,1.615599e-07,0
2,amazon investigating chinese employees selling...,NOT,NOT,0.999991,1.603185e-06,2.925236e-06,0.000004,4.141915e-08,1
3,person someone shouldvetaken piece shit volcano,"OFF, UNT","OFF, UNT",0.000002,2.697569e-04,5.206646e-02,0.000620,9.470415e-01,1
4,person person obama wanted liberals amp illega...,NOT,NOT,0.999974,1.549825e-05,4.031895e-06,0.000006,3.677636e-07,1
...,...,...,...,...,...,...,...,...,...
13235,person sometimes get strong vibes people mans ...,"OFF, TIN, IND",NOT,0.999991,2.503552e-06,1.425680e-06,0.000005,6.626452e-08,0
13236,benidorm creamfields maga shabby summer,NOT,NOT,0.999979,2.891651e-06,3.625252e-06,0.000009,5.847009e-06,1
13237,person report garbage dont give crap,"OFF, TIN, OTH","OFF, TIN, IND",0.000083,1.629019e-03,9.966840e-01,0.000495,1.108795e-03,0
13238,person pussy,"OFF, UNT","OFF, TIN, IND",0.000078,1.542336e-02,8.800029e-01,0.017194,8.730197e-02,0


In [0]:
def calScores(frame):
  lst=['NOT','OFF, TIN, GRP','OFF, TIN, IND','OFF, TIN, OTH','OFF, UNT']
  lstFscore=[]
  for item in lst:
    tp=len(frame[(frame['IsCorrect']==1) & (frame['Predictions']==item)])
    fp=len(frame[(frame['IsCorrect']==0) & (frame['Predictions']==item)])
    #precision=tp/tp+fp
    precision=tp/(tp+fp)
    print("Precision for "+item+" : "+str(precision))
    fn=len(frame[(frame['Original']==item) & (frame['Predictions']!=item)])
    recall=tp/(tp+fn)
    print("Recall for "+item+" : "+str(recall))
    fScore=(2*precision*recall)/(precision+recall)
    lstFscore.append(fScore)
  return lstFscore

tp=len(predicted[(predicted['IsCorrect']==1) & (predicted['Predictions']=='OFF, UNT')])
fp=len(predicted[(predicted['IsCorrect']==0) & (predicted['Predictions']=='OFF, UNT')])
print(tp,fp)
calScores(predicted)

196 327
Precision for NOT : 0.7524561403508772
Recall for NOT : 0.9703619909502262
Precision for OFF, TIN, GRP : 0.46265060240963857
Recall for OFF, TIN, GRP : 0.1787709497206704
Precision for OFF, TIN, IND : 0.7319711538461539
Recall for OFF, TIN, IND : 0.25301204819277107
Precision for OFF, TIN, OTH : 0.17142857142857143
Recall for OFF, TIN, OTH : 0.030379746835443037
Precision for OFF, UNT : 0.37476099426386233
Recall for OFF, UNT : 0.37404580152671757


[0.8476284584980237,
 0.2578912021490934,
 0.37604198826798396,
 0.05161290322580645,
 0.3744030563514804]